# Environment

In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, chromsizes, bigwig, Tileset
import clodius
import os
import os.path as path
import pandas as pd
import itertools
import negspy.coordinates as nc

## Load annotations

In [2]:
genes = beddb("Data/hg38/gene-annotations-hg38.beddb")
chrom_sizes = chromsizes("hg38.chrom.sizes")
label_font_size = 18

chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        tileset=chrom_sizes,
        position="top",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        tileset=chrom_sizes,
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    )
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        tileset=genes,
        position="top",
        height=150,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        tileset=genes,
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    )
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    )
}

hg38 = nc.get_chrominfo("hg38")

# Data
## Contact matrices

In [3]:
metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
metadata = metadata.loc[metadata.Include == "Yes", :]
metadata["SampleID"] = ["PCa" + str(i) for i in metadata["Sample ID"]]

tumour_samples = metadata["SampleID"].tolist()
t2e_samples = metadata.loc[metadata["T2E Status"] == "Yes", "SampleID"].tolist()
nont2e_samples = metadata.loc[metadata["T2E Status"] == "No", "SampleID"].tolist()

benign_metadata = pd.read_csv(    path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "config.tsv"), sep="\t", header=0)
benign_metadata = benign_metadata.loc[~benign_metadata["Sample"].str.startswith("PCa"), :]
benign_samples = benign_metadata["Sample"].tolist()

all_samples = tumour_samples + benign_samples
all_names = pd.DataFrame({"Sample": all_samples, "Name": metadata["Patient ID"].tolist() + benign_samples})
cooler_files = (
    [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in tumour_samples]
    + [path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "Contacts", s + ".mcool") for s in benign_samples]
)
lowc_tilesets = {s: cooler(f) for s, f in zip(all_samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

min_resolution = 5000
heatmap_size = 250
lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": all_names.loc[all_names["Sample"] == s, "Name"].values[0],
            },
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": all_names.loc[all_names["Sample"] == s, "Name"].values[0],
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": all_names.loc[all_names["Sample"] == s, "Name"].values[0],
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": all_names.loc[all_names["Sample"] == s, "Name"].values[0],
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=2 * heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": all_names.loc[all_names["Sample"] == s, "Name"].values[0],
            }
        )
    } for s in all_samples
}

## TADs

In [4]:
windows = list(range(3, 31))

tad_1D_files = {s: {w: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "separated-TADs", s + ".40000bp.w_" + str(w) + ".domains.beddb") for w in windows} for s in tumour_samples}
tad_2D_files = {s: {w: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", "separated-TADs", s + ".40000bp.w_" + str(w) + ".domains.bed2ddb") for w in windows} for s in tumour_samples}
htad_files = {s: path.join("..", "2020-01-15_TAD-aggregation", "resolved-TADs", s + ".40000bp.aggregated-domains.bed2ddb") for s in tumour_samples}

def bed2ddb(filepath, uuid=None, **kwargs):
    from clodius.tiles.utils import tiles_wrapper_2d
    from clodius.tiles.bed2ddb import get_2d_tileset_info, get_2D_tiles
    return Tileset(
        uuid=uuid,
        tileset_info=lambda: get_2d_tileset_info(filepath),
        tiles=lambda tids: tiles_wrapper_2d(
            tids,
            lambda z,x,y: get_2D_tiles(filepath, z, x, y)[(x, y)]
        ),
        **kwargs
    )

tad_1D_tilesets = {s: {w: beddb(v[w], name=s + " TADs (w=" + str(w) +")") for w in windows} for s, v in tad_1D_files.items()}
tad_2D_tilesets = {s: {w: bed2ddb(v[w], name=s + " TADs (w=" + str(w) +")") for w in windows} for s, v in tad_2D_files.items()}
htad_tilesets = {s: bed2ddb(v, name=s + " TADs") for s, v in htad_files.items()}

tads = {
    s: {
        w: {
           "top": Track(
                track_type="bedlike",
                position="top",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
            "bottom": Track(
                track_type="bedlike",
                position="bottom",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
            "left": Track(
                track_type="bedlike",
                position="left",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
            "right": Track(
                track_type="bedlike",
                position="right",
                tileset=tad_1D_tilesets[s][w],
                filetype="beddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
        } for w in windows
    } for s in tumour_samples
}

tad_polygons = {
    s: {
        w: {
           "top": Track(
                track_type="horizontal-2d-rectangle-domains",
                position="top",
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
            "bottom": Track(
                track_type="horizontal-2d-rectangle-domains",
                position='bottom',
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
            "centre": Track(
                track_type="2d-rectangle-domains",
                position="center",
                tileset=tad_2D_tilesets[s][w],
                filetype="bed2ddb",
                options={
                    "showMousePosition": True,
                    "name": "{} (w = {})".format(s, w),
                },
            ),
        } for w in windows
    } for s in tumour_samples
}

hierarchical_tads = {
    s: {
       "top": Track(
            track_type="2d-rectangle-domains",
            position="top",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            },
        ),
        "bottom": Track(
            track_type="2d-rectangle-domains",
            position="bottom",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            },
        ),
        "centre": Track(
            track_type="2d-rectangle-domains",
            position="center",
            tileset=htad_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            }
        ),
    } for s in tumour_samples
}

## H3K27ac ChIP-seq

In [5]:
line_height = 50
chip_files = {s: path.join("..", "..", "Data", "Processed", "2019-05-03_PCa-H3K27ac-peaks", "Peaks", s + "_treat_pileup.filtered.bw") for s in tumour_samples}
chip_tilesets = {s: bigwig(f) for s, f in chip_files.items()}
chip = {
    s: {
        "top": Track(
            track_type="horizontal-line",
            position="top",
            tileset=ts,
            filetype="vector",
            height=line_height,
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "bottom": Track(
            track_type="horizontal-line",
            position="bottom",
            tileset=ts,
            filetype="vector",
            height=line_height,
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "left": Track(
            track_type="vertical-line",
            position="left",
            tileset=ts,
            filetype="vector",
            height=line_height,
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "right": Track(
            track_type="vertical-line",
            position="right",
            tileset=ts,
            filetype="vector",
            height=line_height,
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
    } for s, ts in chip_tilesets.items()
}

peak_files = {s: path.join("..", "..", "Data", "Processed", "2019-05-03_PCa-H3K27ac-peaks", "Peaks", s + "_peaks.filtered.beddb") for s in tumour_samples}
peak_tilesets = {s: beddb(f) for s, f in peak_files.items()}
peaks = {
    s: {
        "top": Track(
            track_type="bedlike",
            position="top",
            tileset=ts,
            filetype="beddb",
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "bottom": Track(
            track_type="bedlike",
            position="bottom",
            tileset=ts,
            filetype="beddb",
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "left": Track(
            track_type="bedlike",
            position="left",
            tileset=ts,
            filetype="beddb",
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
        "right": Track(
            track_type="bedlike",
            position="right",
            tileset=ts,
            filetype="beddb",
            options={
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Patient ID"].values[0],
            },
        ),
    } for s, ts in peak_tilesets.items()
}

## Load breakpoints

In [6]:
BREAK_DIR = path.join("..", "2019-07-24_breakfinder", "Breakpoints", "Default")
breaks = {s: pd.read_csv(
    path.join(BREAK_DIR, s + ".breaks.sorted.manually-resolved.tsv"),
    sep="\t",
    header=None,
    index_col=False,
    names=["chr_x", "start_x", "end_x", "chr_y", "start_y", "end_y", "name", "logodds", "strand_x", "strand_y", "resolution", "Type", "Notes"],
) for s in tumour_samples}

break_files = {s: path.join(BREAK_DIR, s + ".breaks.bed2ddb") for s in tumour_samples}
break_tilesets = {s: bed2ddb(v, name=s + " Breaks") for s, v in break_files.items()}
break_tracks = {
    s: {
       "top": Track(
            track_type="horizontal-2d-rectangle-domains",
            position="top",
            tileset=break_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            },
        ),
        "bottom": Track(
            track_type="vertical-2d-rectangle-domains",
            position="bottom",
            tileset=break_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            },
        ),
        "centre": Track(
            track_type="2d-rectangle-domains",
            position="center",
            tileset=break_tilesets[s],
            filetype="bed2ddb",
            options={
                "showMousePosition": True,
                "name": s,
            }
        ),
    } for s in tumour_samples
}

# Paper figures

In [7]:
fig1_view = [
    View(
        tracks=[
            chrom_labels["top"], gene_annots["top"],
            chip[s]["top"],
            tads[s][3]["top"],
            tads[s][10]["top"],
            tads[s][17]["top"],
            tads[s][24]["top"],
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr14", 36800000, hg38),
            nc.chr_pos_to_genome_pos("chr14", 38400000, hg38),
        ]
    ) for s in tumour_samples
]
display, server, viewconf = higlass.display(
    views=fig1_view,
    server_port=8192
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'aRJSzRvvRbSYYBuuY0eMpw', 'tracks': {'top': [{'ty…

## Effects of structural variants on TAD boundaries

### A structural variant that alter local TAD boundaries

In [21]:
t2e_translocation = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["bottom"],
        lowc_heatmaps["PCa13848"]["centre"],
        lowc_heatmaps["PCa13848"]["left"],
        #hierarchical_tads["PCa13848"]
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)
nont2e_translocation = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["top"],
        lowc_heatmaps["PCa3023"]["centre"],
        lowc_heatmaps["PCa3023"]["left"],
    ],
    initialYDomain=[
        nc.chr_pos_to_genome_pos("chr21", 38500000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 41500000, hg38)
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 33000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 38000000, hg38)
    ],
)

t2e_translocation_insertion = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa13848"]["centre"],
        tads["PCa13848"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)
nont2e_translocation_insertion = View(
    tracks=[
        chrom_labels["left"], gene_annots["left"],
        lowc_heatmaps["PCa3023"]["centre"],
        tads["PCa3023"][3]["top"],
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 34000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 37000000, hg38)
    ],
)
display, server, viewconf = higlass.display(
    views=[
        t2e_translocation, nont2e_translocation,
        t2e_translocation_insertion, nont2e_translocation_insertion
    ],
    value_scale_syncs = [
        [
            (t2e_translocation, lowc_heatmaps["PCa13848"]["centre"]),
            (nont2e_translocation, lowc_heatmaps["PCa3023"]["centre"]),
        ],
        [
            (t2e_translocation_insertion, lowc_heatmaps["PCa13848"]["centre"]),
            (nont2e_translocation_insertion, lowc_heatmaps["PCa3023"]["centre"]),
        ],
    ],
    zoom_syncs=[
        [t2e_translocation, nont2e_translocation],
        [t2e_translocation_insertion, nont2e_translocation_insertion]
    ],
    location_syncs=[
        [t2e_translocation, nont2e_translocation],
        [t2e_translocation_insertion, nont2e_translocation_insertion]
    ],
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'cxWlndDmTyOisb0lZG96dw', 'tracks': {'top': [{'ty…

### A structural variant that does not affect local TAD boundaries

In [8]:
sv_sample_id = "PCa13848"
nonsv_sample_id = "PCa3023"
pos = {
    "x": [
        nc.chr_pos_to_genome_pos("chr12", 69700000, hg38),
        nc.chr_pos_to_genome_pos("chr12", 60900000, hg38)
    ],
    "y": [
        nc.chr_pos_to_genome_pos("chr11", 96630000, hg38),
        nc.chr_pos_to_genome_pos("chr11", 99300000, hg38)
    ]
}
non_affecting_sv = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        #lowc_heatmaps[sv_sample_id]["bottom"],
        #CombinedTrack([lowc_heatmaps[sv_sample_id]["centre"], break_tracks[sv_sample_id]["centre"]]),
        #lowc_heatmaps[sv_sample_id]["centre"],
        #lowc_heatmaps[sv_sample_id]["left"],
        #hierarchical_tads[sv_sample_id]
    ],
    initialYDomain=pos["y"],
    initialXDomain=pos["x"],
)
non_affecting_nonsv = View(
    tracks=[
        chrom_labels["top"], gene_annots["top"],
        chrom_labels["left"], gene_annots["left"],
        #lowc_heatmaps[nonsv_sample_id]["top"],
        #lowc_heatmaps[nonsv_sample_id]["centre"],
        #lowc_heatmaps[nonsv_sample_id]["left"],
    ],
    initialYDomain=pos["y"],
    initialXDomain=pos["x"],
)

display, server, viewconf = higlass.display(
    views=[
        non_affecting_sv, non_affecting_nonsv,
    ],
#    value_scale_syncs = [
#        [
#            (non_affecting_sv, lowc_heatmaps[sv_sample_id]["centre"]),
#            (non_affecting_nonsv, lowc_heatmaps[nonsv_sample_id]["centre"]),
#        ],
#        [
#            (non_affecting_sv, lowc_heatmaps[sv_sample_id]["centre"]),
#            (non_affecting_nonsv, lowc_heatmaps[nonsv_sample_id]["centre"]),
#        ],
#    ],
#    zoom_syncs=[
#        [non_affecting_sv, non_affecting_nonsv],
#    ],
#    location_syncs=[
#        [non_affecting_sv, non_affecting_nonsv],
#    ],
    server_port=8192
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'QbHBZLcmR2OnTYabH3zS7g', 'tracks': {'top': [{'ty…

# Images
## Important prostate cancer regions

In [7]:
display, server, viewconf = higlass.display([
    View(
        tracks=[chrom_labels["top"], gene_annots["top"]],
        initialXDomain=[
            nc.chr_pos_to_genome_pos("chr13", 20000000, hg38),
            nc.chr_pos_to_genome_pos("chr13", 20400000, hg38)
        ],
    )
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'G8NZVn4pR8O8zbgSqXacjA', 'tracks': {'top': [{'ty…

### FOXA1

In [21]:
foxa1_view = View(
    tracks=[
        lowc_heatmaps["PCa13266"]["centre"],
        gene_annots["top"],
        chrom_labels["top"],
        CombinedTrack(
            tracks=[chip[s]["top"] for s in tumour_samples],
            options={
                "height": 150
            }
        )
    ],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 39000000, hg38)
    ],
)

display, server, viewconf = higlass.display(
    [foxa1_view],
    value_scale_syncs=[[(foxa1_view, chip[s]["top"]) for s in tumour_samples]]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'bJyIaQsuSNKaAYmh13ACJQ', 'tracks': {'top': [{'ty…

### _TMPRSS2 - ERG_ fusion

In [44]:
t2e_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]],
        #+ [lowc_heatmaps[s]["top"] for s in t2e_samples]
        #+ [lowc_heatmaps[s]["top"] for s in nont2e_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr21", 36000000, hg38),
        nc.chr_pos_to_genome_pos("chr21", 42000000, hg38)
    ],
)

display, server, viewconf = higlass.display(
    views=[t2e_view]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Ygmvoh7hT0qIWaTlQwEOZA', 'tracks': {'top': [{'ty…

## AR

In [10]:
ar_tracks = [chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples]
ar_view = View(
    tracks=ar_tracks,
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chrX", 65040000, hg38),
        nc.chr_pos_to_genome_pos("chrX", 70200000, hg38)
    ]
)

display, server, viewconf = higlass.display([ar_view], value_scale_syncs = [ar_tracks])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'bd-LlC-qR7CeX1BXv4rzMA', 'tracks': {'top': [{'ty…

## MYC

In [11]:
myc_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr8", 126000000, hg38),
        nc.chr_pos_to_genome_pos("chr8", 130000000, hg38)
    ],
)

display, server, viewconf = higlass.display([myc_view])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'G7cYX6t0TSSkImMeIPftJA', 'tracks': {'top': [{'ty…

## CTCF, YY1, HOXB13

In [12]:
ctcf_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr16", 65000000, hg38),
        nc.chr_pos_to_genome_pos("chr16", 69000000, hg38)
    ],
)
yy1_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr14", 98000000, hg38),
        nc.chr_pos_to_genome_pos("chr14", 102000000, hg38)
    ],
)
hoxb13_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr17", 46160000, hg38),
        nc.chr_pos_to_genome_pos("chr17", 51320000, hg38)
    ],
)

myc_insertion_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [lowc_heatmaps[s]["top"] for s in all_samples],
    initialXDomain=[
        nc.chr_pos_to_genome_pos("chr12", 70480000 - 5e6, hg38),
        nc.chr_pos_to_genome_pos("chr12", 70570000 + 5e6, hg38)
    ],
)

display, server, viewconf = higlass.display([
    myc_insertion_view
])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'c4xLMYmkQem72W4Di51fzA', 'tracks': {'top': [{'ty…

## Structural variants

### Breakpoint pairs

In [22]:
zoom_offset = 1e6
s = "PCa40507"
i = list(range(0, 2))
v = [
    View(
        tracks=[
            chrom_labels["top"], gene_annots["top"],
            chrom_labels["left"], gene_annots["left"]]
            #CombinedTrack([lowc_heatmaps[s]["top"], break_tracks[s]["top"]]),
            #lowc_heatmaps[s]["top"],
            #chip[s]["top"], peaks[s]["top"],
            + [tads[s][w]["top"] for w in [3,10,20,30]]
            + [CombinedTrack([lowc_heatmaps[s]["centre"], break_tracks[s]["centre"]]),
            tads[s][3]["left"],
            #lowc_heatmaps[s]["left"],
            #chip[s]["left"], peaks[s]["left"],
        ],
        initialYDomain=[
            nc.chr_pos_to_genome_pos(breaks[s].iloc[j].chr_x, int(breaks[s].iloc[j].start_x) - zoom_offset, hg38),
            nc.chr_pos_to_genome_pos(breaks[s].iloc[j].chr_x, int(breaks[s].iloc[j].end_x) + zoom_offset, hg38)
        ],
        initialXDomain=[
            nc.chr_pos_to_genome_pos(breaks[s].iloc[j].chr_y, int(breaks[s].iloc[j].start_y) - zoom_offset, hg38),
            nc.chr_pos_to_genome_pos(breaks[s].iloc[j].chr_y, int(breaks[s].iloc[j].end_y) + zoom_offset, hg38)
        ],
    ) for j in i
]
    
display, server, viewconf = higlass.display(
    views=v,
    zoom_syncs=[v],
    server_port=8192
)
#print(breaks[s].iloc[i, :])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'NfVEdIekT4qdE75LnmtnwQ', 'tracks': {'top': [{'ty…

## Differential TAD boundaries

In [16]:
tad_view = View(
    tracks=[chrom_labels["top"], gene_annots["top"]] + [tads[s][3]["top"] for s in tumour_samples],
    initialXDomain = [
        nc.chr_pos_to_genome_pos("chr10", 67570000, hg38),
        nc.chr_pos_to_genome_pos("chr10", 69950000, hg38),
    ]
)

display, server, viewconf = higlass.display(views=[tad_view], server_port=8192)
print(tumour_samples)
display

['PCa13266', 'PCa13848', 'PCa14121', 'PCa19121', 'PCa3023', 'PCa33173', 'PCa40507', 'PCa51852', 'PCa53687', 'PCa56413', 'PCa57054', 'PCa57294', 'PCa58215']


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'Vl2vaQV0Q8O4v3Qw-WiXvw', 'tracks': {'top': [{'ty…

## Complex events

In [ ]:
s = "Benign-Prostate-3330581"
pos = {
    "x": [
        nc.chr_pos_to_genome_pos("chr19", 36300000, hg38),
        nc.chr_pos_to_genome_pos("chr19", 44600000, hg38)
    ],
    "y": [
        nc.chr_pos_to_genome_pos("chr19", 56300000, hg38),
        nc.chr_pos_to_genome_pos("chr19", 58400000, hg38)
    ]
}
v = [
    #View(
    #    tracks=[
    #        chrom_labels["top"], gene_annots["top"],
    #        chrom_labels["left"], gene_annots["left"],
    #        lowc_heatmaps[s]["centre"],
    #    ],
    #    initialXDomain=pos["x"],
    #),
    View(
        tracks=[
            chrom_labels["top"], gene_annots["top"],
            chrom_labels["left"], gene_annots["left"],
            lowc_heatmaps[s]["centre"],
        ],
        initialXDomain=pos["x"],
        initialYDomain=pos["y"],
    ),
    #View(
    #    tracks=[
    #        #chrom_labels["top"], gene_annots["top"],
    #        #chrom_labels["left"], gene_annots["left"],
    #        lowc_heatmaps[s]["centre"],
    #    ],
    #    initialXDomain=pos["y"],
    #),
    #View(
    #    tracks=[
    #        chrom_labels["top"], gene_annots["top"],
    #        chrom_labels["left"], gene_annots["left"],
    #        lowc_heatmaps[s]["centre"],
    #        break_tracks[s]["centre"],
    #    ],
    #    initialXDomain=[
    #        nc.chr_pos_to_genome_pos("chr16", 60000000, hg38),
    #        nc.chr_pos_to_genome_pos("chr16", 63000000, hg38)
    #    ],
    #)
]

display, server, viewconf = higlass.display(
    views=v,
    server_port=8192
)
display